In [3]:
import sys
import re
import os
current_dir = os.getcwd()
ds_path = os.path.join(os.path.dirname(current_dir), 'ds')
sys.path.append(ds_path)

from generate_loader import dataloaders
from torchvision import transforms
from torch.nn import functional as F
from torch import nn
import torch

In [4]:
trainloader, testloader = dataloaders()

In [5]:
trainloader

In [6]:
testloader

In [7]:
class BoxRegressorCNN(nn.Module):
    def __init__(self):
        super(BoxRegressorCNN, self).__init__()
        self.features_ext = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1), # 720/2=360, 1160/2=580
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1), # 360/2=180, 580/2=290
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1), # 180/2=90, 290/2=145
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1), # 90/2=45, 145/2=73
            nn.ReLU(),
            )
        

        self.linear = nn.Sequential(
            nn.Linear(256*45*73, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            
            )
        
        self.result = nn.Linear(512, 4)
        

    def forward(self, x):
        x = self.features_ext(x)
        x = torch.flatten(x, 1) 
        x = self.linear(x)# flatten all dimensions except batch
        out = torch.sigmoid(self.result(x))
        return out
    
model = BoxRegressorCNN()

In [8]:
criterion = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [9]:
def train_model(model, trainloader, testloader, epochs=10, device="cpu"):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for imgs, targets in trainloader:
            imgs, targets = imgs.to(device), targets.to(device)

            preds = model(imgs)            # [B, 4]
            loss = criterion(preds, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * imgs.size(0)

        # середній train loss
        train_loss /= len(trainloader.dataset)

        # валідація
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for imgs, targets in testloader:
                imgs, targets = imgs.to(device), targets.to(device)
                preds = model(imgs)
                loss = criterion(preds, targets)
                val_loss += loss.item() * imgs.size(0)

        val_loss /= len(testloader.dataset)

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    return model

In [10]:
train_model(model, trainloader, testloader, epochs=10)

KeyboardInterrupt: 